In [ ]:
# Import neuron
from neuron import h,gui

# Import Mechanisms
import os
current_directory = os.getcwd()
print(current_directory)
path = os.path.join(current_directory, "mechanisms", "nrnmech.dll")
print(path)
h.nrn_load_dll(path)

# Import utilities 
from neuron.units import ms,mV,um,V,m 
import matplotlib.pyplot as plt 
import numpy as np 
h.load_file("stdrun.hoc")
h.cvode_active(0) #turn off variable time step
# h.load_file("anatscale.hoc")
import pandas as pd


#Local Field Potential calculation
#h.load_file("./functions/field.hoc")

##### Import cell model
from functions.ballandstick import BallAndStick,BallAndStick_pas,BallAndStick2
cell=BallAndStick2(0,0,0,0,0,0)
print(cell.soma.nseg,cell.dend.nseg)

# Interpolate and set pointers between xtra and extracellular
h.load_file("./functions/interpxyz.hoc")
h.load_file("./functions/setpointers.hoc")




from functions.calcrx import setelec,homogenous,set_uniform_field_between_plates

# plot=setelec([50],[50],[50])

v_plate=40*V #- potential difference between the plates
distance=1*m #distance
field_orientation=np.array([1,0,0])#along the x axis
ref_point=[0,0,0] #reference point with a 0 e_extracellular

set_uniform_field_between_plates(v_plate,distance,field_orientation,ref_point)


dt=1*ms
simtime=1000*ms
depth=0
modfreq=0
v_init=-65*mV

h.dt=dt
h.v_init=v_init


import functions.stim as stim
ton=20
amp=3000
dur=900
freq=10
depth=1
modfreq=0
times,stim1=stim.ampmodulation_wiki(ton,amp,depth,dt,dur,simtime,freq,modfreq)

simparams=[dt,simtime]
stimparams=[v_plate,ton,amp,dur,freq,depth,modfreq]
run_id=0

#Save parameters to a file
from functions.savedata import saveparams
folder=saveparams(run_id,cell,simparams,stimparams)

# test with IClamp
teststim=h.IClamp(cell.soma(0.5))
teststim.delay=1
teststim.amp=100
teststim.dur=100

teststim2=h.IClamp(cell.soma(1))
teststim2.delay=500
teststim2.amp=500
teststim2.dur=250

t=h.Vector().record(h._ref_t)
is_xtra=h.Vector().record(h._ref_is_xtra)
soma_v=h.Vector().record(cell.soma(0.5)._ref_v)
dend_v=h.Vector().record(cell.dend(0.5)._ref_v)
extracellular_v_soma = h.Vector().record(cell.soma(0.5)._ref_vext[0])

from functions.savedata import save_locations
save_locations(folder,cell)

#Attempt1
# from functions.shapeplot_movie import movie_v

# callback=movie_v(cell)

# h.load_file("movierun.hoc")

# h.movierun()

#Attempt 2 
# from functions.gpt_shapeplot import morphology_voltage_movie
# # fig, callback = morphology_voltage_movie(cell)
# tmin=10
# tmax=50
# fig, callback=morphology_voltage_movie(cell, folder,tmin=tmin, tmax=tmax)
# from functions.gpt_shapeplot import update_plot,plot_final

# v_over_time,callback= update_plot(cell)


#Attempt 3
# h.finitialize(v_init)
# h.continuerun(simtime)
# print(v_over_time)
# fig=plot_final(cell,v_over_time)
# fig.write_html("./data/final_voltage_plot.html")

# Attempt 4

ps=h.PlotShape(True)
ps.variable("v")
h.fast_flush_list.append(ps)
ps.exec_menu("Shape Plot")
ps.exec_menu("Show Diam")

from functions.shapeplot_movie import plot_view_mark,shape_plot,SP_highlight, plot_view_color

ps=plot_view_mark()
fig=shape_plot(cell,"rx_xtra")
SP_highlight(cell)
ps.plot(plt)
# plt.show()
tmin=100
tmax=120
results={}
ps,callback=plot_view_color(folder,cell,results,tmin,tmax)


c:\Users\Pc\Documents\Charite\NEURON\Extracellular_test\Homogeneous_Efield
c:\Users\Pc\Documents\Charite\NEURON\Extracellular_test\Homogeneous_Efield\mechanisms\nrnmech.dll
1 151
After any change to cell geometry or nseg, be sure to invoke setpointers()
c:\Users\Pc\Documents\Charite\NEURON\Extracellular_test\Homogeneous_Efield


0.0

In [2]:
# import functions.calculate_max as calcmax
# results,file, callback = calcmax.master_callback(cell, folder, run_id,1)
import functions.maxv_shplot as maxshplot
max_shift, callback=maxshplot.max_shift(cell)
# from functions.shapeplot_movie import save_changing_rvp
# tmin=10
# tmax=20
# results_file=os.path.join(folder,"results.csv")
# results=pd.read_csv()
# callback=save_changing_rvp(cell=cell,results=results,folder=folder,tmin=tmin,tmax=tmax)

In [5]:
h.load_file("movierun.hoc")
h.movierun(simtime)

# h.finitialize(v_init)
# h.continuerun(simtime)
# # file.close()

0.0

In [ ]:
# print(results)

# from functions.shapeplot_movie import save_changing_rvp
# tmin=10
# tmax=20
# # results_file=os.path.join(folder,"results.csv")
# # results=pd.read_csv()
# callback=save_changing_rvp(cell=cell,results={},folder=folder,tmin=tmin,tmax=tmax)

{'max_v': 95.35448581039788, 'min_v': 0.0, 'max_seg': BallAndStick2[0].dend2(0.996689), 'min_seg': BallAndStick2[0].soma(0.5), 'max_sec': BallAndStick2[0].dend2, 'min_sec': BallAndStick2[0].soma}
minseg =BallAndStick2[0].soma(0.5)
 maxseg=BallAndStick2[0].dend2(0.996689)


In [ ]:
# import functions.after_movieplot as after_plt

# after_plt.plot_voltage_shape(folder,cell,max_shift)
# h.finitialize(v_init)
# h.continuerun(simtime)

0.0

In [6]:
# plt.Figure()
# plt.plot(t,soma_v,label="soma")
# plt.plot(t,dend_v,label="dend")
# plt.xlabel("time(ms)")
# plt.ylabel("Membrane Voltage (mV)") #vint-vext~
# plt.legend()
# plt.show()